<a href="https://colab.research.google.com/github/drvoss/Colab-Notebooks/blob/master/RNN_encoder_decoder_EnglishToSpainish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip

--2019-03-20 12:41:37--  http://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2819791 (2.7M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.69M  10.6MB/s    in 0.3s    

2019-03-20 12:41:37 (10.6 MB/s) - ‘spa-eng.zip’ saved [2819791/2819791]

Archive:  spa-eng.zip
  inflating: _about.txt              
  inflating: spa.txt                 


In [0]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset,DataLoader,TensorDataset)
import tqdm

import re
import collections
import itertools
remove_marks_regex = re.compile(
"[\,\(\)\[\]\*:;¿¡]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")
unk = 0
sos = 1
eos = 2
def normalize(text):
  text = text.lower()
  # 불필요한 문자 제거
  text = remove_marks_regex.sub("", text)
  # ?!.와 단어 사이에 공백 삽입
  text = shift_marks_regex.sub(r" \1", text)
  return text

def parse_line(line):
  line = normalize(line.strip())
  # 번역 소스(src)와 번역 타깃(trg) 각각의 토큰을 리스트로 만든다
  src, trg = line.split("\t")
  src_tokens = src.strip().split()
  trg_tokens = trg.strip().split()
  return src_tokens, trg_tokens

def build_vocab(tokens):
  # 파일 안의 모든 문장에서 토큰의 등장 횟수를 확인
  counts = collections.Counter(tokens)
  # 토큰의 등장 횟수를 많은 순으로 나열
  sorted_counts = sorted(counts.items(),
  key=lambda c: c[1], reverse=True)
  # 세 개의 태그를 추가해서 정방향 리스트와 역방향 용어집 만들기
  word_list = ["<UNK>", "<SOS>", "<EOS>"] \
  + [x[0] for x in sorted_counts]
  word_dict = dict((w, i) for i, w in enumerate(word_list))
  return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
  # 끝에 종료 태그를 붙임
  words = words + ["<EOS>"]
  # 사전을 이용해서 수치 리스트로 변환
  words = [word_dict.get(w, 0) for w in words]
  seq_len = len(words)
  # 길이가 max_len 이하이면 패딩한다
  if seq_len < max_len + 1:
    words = words + [padding] * (max_len + 1 - seq_len)
  # 텐서로 변환해서 반환
  return torch.tensor(words, dtype=torch.int64), seq_len

class TranslationPairDataset(Dataset):
    def __init__(self, path, max_len=15):
        # 단어 수사 많은 문장을 걸러내는 함수
        def filter_pair(p):
            return not (len(p[0]) > max_len 
                        or len(p[1]) > max_len)
        # 파일을 열어서, 파스 및 필터링       
        with open(path) as fp:
            pairs = map(parse_line, fp)
            pairs = filter(filter_pair, pairs)
            pairs = list(pairs)
        # 문장의 소스와 타켓으로 나눔
        src = [p[0] for p in pairs]
        trg = [p[1] for p in pairs]
        #각각의 어휘집 작성
        self.src_word_list, self.src_word_dict = \
            build_vocab(itertools.chain.from_iterable(src))
        self.trg_word_list, self.trg_word_dict = \
            build_vocab(itertools.chain.from_iterable(trg))
        # 어휘집을 사용해서 Tensor로 변환
        self.src_data = [words2tensor(
            words, self.src_word_dict, max_len)
                for words in src]
        self.trg_data = [words2tensor(
            words, self.trg_word_dict, max_len, -100)
                         for words in trg]
    def __len__(self):
        return len(self.src_data)
      
    def __getitem__(self, idx):
        src, lsrc = self.src_data[idx]
        trg, ltrg = self.trg_data[idx]
        return src, lsrc, trg, ltrg
      
batch_size = 64
max_len = 10
path = "/content/spa.txt"
ds = TranslationPairDataset(path, max_len=max_len)
loader = DataLoader(ds, batch_size=batch_size, shuffle=True,
                    num_workers=4)

class Encoder(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                  hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, 
          embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        
    def forward(self, x, h0=None, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        _, h = self.lstm(x, h0)
        return h
      
class Decoder(nn.Module):
    def __init__(self, num_embeddings,
                 embedding_dim=50, 
                 hidden_size=50,
                 num_layers=1,
                 dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_size,
                            num_layers, batch_first=True,
                            dropout=dropout)
        self.linear = nn.Linear(hidden_size, num_embeddings)
    
    def forward(self, x, h, l=None):
        x = self.emb(x)
        if l is not None:
            x = nn.utils.rnn.pack_padded_sequence(
                x, l, batch_first=True)
        x, h = self.lstm(x, h)
        if l is not None:
            x = nn.utils.rnn.pad_packed_sequence(x, batch_first=True, padding_value=0)[0]
        x = self.linear(x)
        return x, h
      
def translate(input_str, enc, dec, max_len=15, device="cpu"):
    # 입력 문자열을 수치화해서 Tensor로 변환
    words = normalize(input_str).split()
    input_tensor, seq_len = words2tensor(words, 
        ds.src_word_dict, max_len=max_len)
    input_tensor = input_tensor.unsqueeze(0)
    # 엔코더에서 사용하므로 입력값의 길이도 리스트로 만들어둔다
    seq_len = [seq_len]
    # 시작 토큰 준비
    sos_inputs = torch.tensor(sos, dtype=torch.int64)
    input_tensor = input_tensor.to(device)
    sos_inputs = sos_inputs.to(device)
    # 입력 문자열을 엔코더에 넣어서 컨텍스트 얻기
    ctx = enc(input_tensor, l=seq_len)
    # 시작 토큰과 컨텍스트를 디코더의 초깃값으로 설정
    z = sos_inputs
    h = ctx
    results = []
    for i in range(max_len):
        # Decoder로 다음 단어 예측
        o, h = dec(z.view(1, 1), h)
        # 선형 계층의 출력이 가장 큰 위치가 다음 단어의 ID
        wi = o.detach().view(-1).max(0)[1]
        if wi.item() == eos:
            break
        results.append(wi.item())
        # 다음 입력값으로 현재 출력 ID를 사용
        z = wi
    # 기록해둔 출력 ID를 문자열로 변환
    return " ".join(ds.trg_word_list[i] for i in results)

''''' 테스트
enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
translate("I am a student.", enc, dec)
'''''

enc = Encoder(len(ds.src_word_list), 100, 100, 2)
dec = Decoder(len(ds.trg_word_list), 100, 100, 2)
enc.to("cuda:0")
dec.to("cuda:0")
opt_enc = optim.Adam(enc.parameters(), 0.002)
opt_dec = optim.Adam(dec.parameters(), 0.002)
loss_f = nn.CrossEntropyLoss()

from statistics import mean

def to2D(x):
    shapes = x.shape
    return x.reshape(shapes[0] * shapes[1], -1)
  
for epoc in range(30):
    # 신경망을 훈련 모드로 설정
    enc.train(), dec.train()
    losses = []
    for x, lx, y, ly in tqdm.tqdm(loader):
        # x의 PackedSequence를 만들기 위해 번역 소스의 길이로 내림차순 정렬한다
        lx, sort_idx = lx.sort(descending=True)
        x, y, ly = x[sort_idx], y[sort_idx], ly[sort_idx]
        x, y = x.to("cuda:0"), y.to("cuda:0")
        # 번역 소스를 엔코더에 넣어서 컨텍스트를 얻는다
        ctx = enc(x, l=lx)
        # y의 PackedSequence를 만들기 위해 번역 소스의 길이로 내림차순 정렬
        ly, sort_idx = ly.sort(descending=True)
        y = y[sort_idx]
        # Decoder의 초깃값 설정
        h0 = (ctx[0][:, sort_idx, :], ctx[1][:, sort_idx, :])
        z = y[:, :-1].detach()
        # -100인 상태에선 Embedding 계산에서 오류가 발생하므로 0으로 변경
        z[z==-100] = 0
        # 디코더에 넣어서 손실 함수 계산
        o, _ = dec(z, h0, l=ly-1)
        loss = loss_f(to2D(o[:]), to2D(y[:, 1:max(ly)]).squeeze())
        # Backpropagation(오차 역전파 실행)
        enc.zero_grad(), dec.zero_grad()
        loss.backward()
        opt_enc.step(), opt_dec.step()
        losses.append(loss.item())
    # 전체 데이터의 계산이 끝나면 현재의
    # 손실 함수 값이나 번역 결과를 표시
    enc.eval(), dec.eval()
    print(epoc, mean(losses))
    with torch.no_grad():
        print(translate("I am a student.",
                         enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("He likes to eat pizza.",
                         enc, dec, max_len=max_len, device="cuda:0"))
        print(translate("She is my mother.",
                         enc, dec, max_len=max_len, device="cuda:0"))